In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregar os dados
df  = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando todos os dataframes
df_all = df.append(test)
df_all.shape

In [ ]:
# Quais colunas do df são do tipo object
df_all.select_dtypes('object').head()


In [ ]:
# Analisando se tem yes para edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Transformando no em zero e yes em 1
# nas colunas edjefa e edjefe
# e transfoma de objct para number
mapeamento = {'yes':1, 'no':0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

df_all.select_dtypes('object').head()

In [ ]:
# Analisando se tem yes para dependency
df_all['dependency'].value_counts()

In [ ]:
# Transformando no em zero e yes em 1
# nas coluna dependency
# e transfoma de objct para number
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)


In [ ]:
df_all.select_dtypes('object').head()

In [ ]:
# Analisando se tem yes para dependency
df_all['dependency'].value_counts()

In [ ]:
df_all['Target'].value_counts()

In [ ]:
df_all.isnull().sum()

# Modelo de arvore inputa outlier 

# Preencher -1 para os nulos
df_all.fillna(-1, inplace=True)

In [ ]:
# train tem respostas
train = df_all[df_all['Target'] != -1]
train.shape

In [ ]:
# test não tem respostas
test = df_all[df_all['Target'] == -1]
test.shape

In [ ]:
train['Target'].value_counts(normalize=True)

In [ ]:
# 1 Transformar os dados
# 2 Separar os dataframes
# 3 Não vamos mais separar as bases em treino e validação - usar out of bag (oob-score)
# 4 instanciar o modelo RF
# 5 Separar as colunas para usar no treino
# 6 Treinar meu modelo


In [ ]:
# Definindo as features / colunas de entrada
# Target é a variável resposta
feats = [c for c in train.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Treinando o modelo
# Importando o RandomForest
# Os registros faltantes são utilizados para realização da validação
from sklearn.ensemble import RandomForestClassifier

# Instanciando o modelo
rf = RandomForestClassifier (n_jobs=-1, oob_score=True, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo 
# Colunas que fazem parte do treino
# Variável resposta - train['Target']
rf.fit(train[feats], train['Target'])

In [ ]:
# Fazendo as previsoes usando o modelo
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Verificando os resultados
test['Target'].value_counts(normalize=True)

In [ ]:
# Gerando o arquivo para submissao
test[['Id','Target']].to_csv('submission.csv', index=False)

In [ ]:
# Limitar o treinamento ao Chefe de família

# Criando um novo dataframe
heads = train[train['parentesco1'] == 1]

In [ ]:
# Criando um novo modelo
rf2 = RandomForestClassifier (n_jobs=-1, oob_score=True, n_estimators=200, random_state=42)

In [ ]:
# Terinando o modleo
rf2.fit(train[feats], train['Target'])

In [ ]:
heads.shape

In [ ]:
# Fazendo as previsoes usando o modelo
test['Target'] = rf2.predict(test[feats]).astype(int)

In [ ]:
# Gerando o arquivo para submissao
test[['Id','Target']].to_csv('submission2.csv', index=False)

In [ ]:
# Feature Enginneering

# Vamos criar coluna para valores percapita
df_all['hsize-pc'] = df_all['hhsize'] / df_all['tamviv']
df_all['phone-pc'] = df_all['qmobilephone'] / df_all['tamviv']
df_all['tablets-pc'] = df_all['v18q1'] / df_all['tamviv']
df_all['rooms-pc'] = df_all['rooms'] / df_all['tamviv']
df_all['rent-pc'] = df_all['v2a1'] / df_all['tamviv']


In [ ]:
# Separando o datafrema em train e test
train, test = df_all[df_all['Target'] != -1],df_all[df_all['Target'] == 1]


In [ ]:
# Definindo as features / colunas de entrada
# Target é a variável resposta
feats = [c for c in train.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Criando um novo modelo
rf3 = RandomForestClassifier (n_jobs=-1, oob_score=True, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo 
# Colunas que fazem parte do treino
# Variável resposta - train['Target']
rf3.fit(train[feats], train['Target'])

In [ ]:
# Fazendo as previsoes usando o modelo
test['Target'] = rf3.predict(test[feats]).astype(int)

In [ ]:
# Gerando o arquivo para submissao
test[['Id','Target']].to_csv('submission.csv', index=False)

In [ ]:
# 
heads2 = train[train['parentesco1'] == 1]

In [ ]:
rf3.fit(heads2[feats], heads2['Target'])

In [ ]:
# Gerando o arquivo para submissao
test[['Id','Target']].to_csv('submission.csv', index=False)

In [ ]:
# Feature Importence
pd.Series(rf3.feature_importances_, index=feats).sort_values().plot.barh(figsize = (20, 40))

In [ ]:
# Criando um novo modelo
rf4 = RandomForestClassifier (max_depth=None, n_jobs=4, oob_score=True, n_estimators=700, random_state=42,
                             min_impurity_decrease=1e-3, min_samples_leaf=2, verbose=0, class_weight = 'balanced')

In [ ]:
# Fazendo as previsoes usando o modelo
test['Target'] = rf4.predict(test[feats]).astype(int)

In [ ]:
# Copiando o campeão
rf4.fit(heads2[feats], heads2['Target'])

In [ ]:
# Gerando o arquivo para submissao
test[['Id','Target']].to_csv('submission.csv', index=False)